# Imports

In [1]:
import gspread_pandas
import pandas as pd

# Load data

In [2]:
spread = gspread_pandas.Spread("search_retired1.profile_export.gsheet")
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(6, 62)


,linkedinProfileUrl,email,linkedinProfile,headline,location,imgUrl,firstName,lastName,fullName,subscribers,...,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,description,twitter,twitterProfileUrl,website
0,https://www.linkedin.com/in/richard-saunders-9...,,https://www.linkedin.com/in/richard-saunders-9...,Retired from Federal Service.,"Silver Spring, Maryland, United States",https://media.licdn.com/dms/image/D4E03AQHxFXh...,Richard,Saunders,Richard Saunders,683,...,62,Information Security Management,54,https://www.linkedin.com/in/richard-saunders-9...,richard-saunders-93876b7,2023-07-14T12:39:37.674Z,,,,
1,https://www.linkedin.com/in/scottalawrence/,,https://www.linkedin.com/in/scottalawrence/,"Senior Manager, Software Engineering at Capit...","Silver Spring, Maryland, United States",https://media.licdn.com/dms/image/C4E03AQHI9E9...,Scott,L.,Scott L.,967,...,15,SDLC,11,https://www.linkedin.com/in/scottalawrence,scottalawrence,2023-07-14T12:40:02.640Z,Experienced software engineering leader with a...,,,
2,https://www.linkedin.com/in/perrycoleman1/,,https://www.linkedin.com/in/perrycoleman1/,"IT Specialist, MBA, MSCIS, PMP, CISSP, DML","Gwynn Oak, Maryland, United States",https://media.licdn.com/dms/image/C4E03AQEJYFZ...,Perry M,Coleman,Perry M Coleman,281,...,5,Military,9,https://www.linkedin.com/in/perrycoleman1,perrycoleman1,2023-07-14T12:40:23.809Z,"Skilled in the areas of Cyber Security, Projec...",,,
3,https://www.linkedin.com/in/kathleenmbovard/,,https://www.linkedin.com/in/kathleenmbovard/,"Skilled career coach, facilitator and mentor c...","Baltimore, Maryland, United States",https://media.licdn.com/dms/image/C4E03AQHfUCO...,Kathleen,Bovard,Kathleen Bovard,,...,,,,https://www.linkedin.com/in/kathleenmbovard,kathleenmbovard,2023-07-14T12:40:42.210Z,,,,
4,https://www.linkedin.com/in/christopher-a-tayl...,,https://www.linkedin.com/in/christopher-a-tayl...,SCORE Mentor & Retired Head of Healthcare & Fi...,"Baltimore, Maryland, United States",https://media.licdn.com/dms/image/C4D03AQEXytm...,Christopher A.,Taylor,Christopher A. Taylor,7423,...,34,Casualty,8,https://www.linkedin.com/in/christopher-a-tayl...,christopher-a-taylor-63002714,2023-07-14T12:41:07.132Z,"Leadership, Public Speaking,Healthcare Industr...",usz8je0,https://twitter.com/usz8je0,zurichna.com/en/industries/healthcare


In [3]:
df.columns

Index(['linkedinProfileUrl', 'email', 'linkedinProfile', 'headline',
       'location', 'imgUrl', 'firstName', 'lastName', 'fullName',
       'subscribers', 'connectionDegree', 'vmid', 'userId',
       'linkedinSalesNavigatorUrl', 'connectionsCount', 'connectionsUrl',
       'mutualConnectionsUrl', 'company', 'companyUrl', 'jobTitle',
       'jobDescription', 'jobLocation', 'jobDateRange', 'jobDuration',
       'company2', 'companyUrl2', 'jobTitle2', 'jobDescription2',
       'jobLocation2', 'jobDateRange2', 'jobDuration2', 'school', 'schoolUrl',
       'schoolDegree', 'schoolDescription', 'schoolDateRange', 'school2',
       'schoolUrl2', 'schoolDegree2', 'schoolDescription2', 'schoolDateRange2',
       'birthday', 'allSkills', 'skill1', 'endorsement1', 'skill2',
       'endorsement2', 'skill3', 'endorsement3', 'skill4', 'endorsement4',
       'skill5', 'endorsement5', 'skill6', 'endorsement6', 'baseUrl',
       'profileId', 'timestamp', 'description', 'twitter', 'twitterProfileUrl',


In [4]:
for col in df.columns:
    print(col)
    print(df.iloc[0][col])
    print()

linkedinProfileUrl
https://www.linkedin.com/in/richard-saunders-93876b7/

email


linkedinProfile
https://www.linkedin.com/in/richard-saunders-93876b7/

headline
Retired from Federal Service.

location
Silver Spring, Maryland, United States

imgUrl
https://media.licdn.com/dms/image/D4E03AQHxFXhV_gMJFg/profile-displayphoto-shrink_800_800/0/1683504155825?e=1694649600&v=beta&t=IQUDsyV7oiWvMxjNJCCE_kTO-3_YJKZZ_bwACnppXAk

firstName
Richard

lastName
Saunders

fullName
Richard Saunders

subscribers
683

connectionDegree
Out of Network

vmid
ACoAAAFbuXwBJ--KuOaKYTPBM9TAthDnrps2ODY

userId
22788476

linkedinSalesNavigatorUrl
https://www.linkedin.com/sales/people/ACoAAAFbuXwBJ--KuOaKYTPBM9TAthDnrps2ODY,name

connectionsCount
500

connectionsUrl
https://www.linkedin.com/search/results/people/?facetConnectionOf=%5B%22ACoAAAFbuXwBJ--KuOaKYTPBM9TAthDnrps2ODY%22%5D&facetNetwork=%5B%22F%22%2C%22S%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH

mutualConnectionsUrl
https://www.linkedin.com/search/results/

# Filter data

In [5]:
# Filter by keywords.
keywords = ["volunteer", "adjunct", "consult"]
prev_len = len(df)
dfs_filtered = []
for col in [
    "headline",
    "jobTitle",
    "jobDescription",
    "jobTitle2",
    "jobDescription2",
    "allSkills",
]:
    dfs_filtered.append(
        df[df[col].apply(lambda x: any(kw in x.lower() for kw in keywords))]
    )
df = pd.concat(dfs_filtered).drop_duplicates().reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)

Dropped 3 rows (50.0%)


In [6]:
print(df.shape)
df

(3, 62)


,linkedinProfileUrl,email,linkedinProfile,headline,location,imgUrl,firstName,lastName,fullName,subscribers,...,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,description,twitter,twitterProfileUrl,website
0,https://www.linkedin.com/in/dr-donald-jg-chiar...,,https://www.linkedin.com/in/dr-donald-jg-chiar...,CEO Chiarella Consulting,"Hanover, Maryland, United States",https://media.licdn.com/dms/image/C5603AQGZXgW...,Dr. Donald JG,"Chiarella, CPEM, CHPP, CISM, CDP, CSM, ITIL,...","Dr. Donald JG Chiarella, CPEM, CHPP, CISM, CD...",907,...,2,Published Author,1,https://www.linkedin.com/in/dr-donald-jg-chiar...,dr-donald-jg-chiarella-cpem-chpp-cism-cdp-csm-...,2023-07-14T12:41:33.388Z,,,,lulu.com/donchiarella
1,https://www.linkedin.com/in/richard-saunders-9...,,https://www.linkedin.com/in/richard-saunders-9...,Retired from Federal Service.,"Silver Spring, Maryland, United States",https://media.licdn.com/dms/image/D4E03AQHxFXh...,Richard,Saunders,Richard Saunders,683,...,62,Information Security Management,54,https://www.linkedin.com/in/richard-saunders-9...,richard-saunders-93876b7,2023-07-14T12:39:37.674Z,,,,
2,https://www.linkedin.com/in/kathleenmbovard/,,https://www.linkedin.com/in/kathleenmbovard/,"Skilled career coach, facilitator and mentor c...","Baltimore, Maryland, United States",https://media.licdn.com/dms/image/C4E03AQHfUCO...,Kathleen,Bovard,Kathleen Bovard,,...,,,,https://www.linkedin.com/in/kathleenmbovard,kathleenmbovard,2023-07-14T12:40:42.210Z,,,,


# Save filtered data to gsheet

In [7]:
# A Google sheet with this name should already exist on the drive.
spread2 = gspread_pandas.Spread(
    "search_retired1.profile_export.filtered.gsheet",
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)